## LEITURA DO ARQUIVO

In [1]:
import pandas as pd

In [2]:
intervalo_das_colunas = [(0,2), (4,6), (8, 10), (11, 13), (14,15), (16,18), (19,21), (22,23), (24,34)]
arquivo = pd.read_fwf('DP.txt', colspecs=intervalo_das_colunas, skiprows=4, index=False)

In [3]:
# seleção de colunas de interesse
arquivo_filtrado = arquivo[['ss','di','hi','m','Demanda']]

# remove coluna desnecessária
arquivo_filtrado = arquivo_filtrado.drop(0)

# Visualização
arquivo_filtrado.head(3)

,ss,di,hi,m,Demanda
1,1,2,0,0,36703
2,1,2,0,1,35673
3,1,2,1,0,34527


## TRATAMENTO DO DATAFRAME

In [4]:
# The names of the columns have to be year, month, day, hour, minute and second: -> padrão para conversão em datetime mais a frente
# Minimal columns are year, month and day
arquivo_filtrado.columns = ["subsistema", "day", "hour","minute", "demanda"] # Renomeação de colunas
arquivo_filtrado.head(3)

,subsistema,day,hour,minute,demanda
1,1,2,0,0,36703
2,1,2,0,1,35673
3,1,2,1,0,34527


In [5]:
import pendulum

hoje = pendulum.today('America/Sao_Paulo')

In [6]:
# No arquivo só temos a informação de dia, hora e minuto -> então usei ano e mês do dia atual 03/10/2021
arquivo_filtrado['year'] = hoje.format('YYYY')
arquivo_filtrado['month'] = hoje.format('MM')
arquivo_filtrado.head(3)

,subsistema,day,hour,minute,demanda,year,month
1,1,2,0,0,36703,2021,10
2,1,2,0,1,35673,2021,10
3,1,2,1,0,34527,2021,10


In [7]:
# Conversão da coluna minute de booleano para inteiro
arquivo_filtrado.loc[arquivo_filtrado['minute'] == '1', 'minute'] = 30
arquivo_filtrado.loc[arquivo_filtrado['minute'] == '0', 'minute'] = 0
arquivo_filtrado.head(3)

,subsistema,day,hour,minute,demanda,year,month
1,1,2,0,0,36703,2021,10
2,1,2,0,30,35673,2021,10
3,1,2,1,0,34527,2021,10


In [8]:
# subsistema -> conversão de código para sigla
arquivo_filtrado.loc[arquivo_filtrado['subsistema'] == '1', 'subsistema'] = 'seco'
arquivo_filtrado.loc[arquivo_filtrado['subsistema'] == '2', 'subsistema'] = 's'
arquivo_filtrado.loc[arquivo_filtrado['subsistema'] == '3', 'subsistema'] = 'ne'
arquivo_filtrado.loc[arquivo_filtrado['subsistema'] == '4', 'subsistema'] = 'n'
arquivo_filtrado.loc[arquivo_filtrado['subsistema'] == '11', 'subsistema'] = 'imperatriz'
arquivo_filtrado.head(3)

,subsistema,day,hour,minute,demanda,year,month
1,seco,2,0,0,36703,2021,10
2,seco,2,0,30,35673,2021,10
3,seco,2,1,0,34527,2021,10


In [9]:
# cria coluna data no formato datetime
arquivo_filtrado['data'] = pd.to_datetime(arquivo_filtrado[['year', 'month', 'day', 'hour', 'minute']])
arquivo_filtrado.head(3)

,subsistema,day,hour,minute,demanda,year,month,data
1,seco,2,0,0,36703,2021,10,2021-10-02 00:00:00
2,seco,2,0,30,35673,2021,10,2021-10-02 00:30:00
3,seco,2,1,0,34527,2021,10,2021-10-02 01:00:00


## Geração de string formatada

In [10]:
from pandas import DataFrame
import sys

def gera_modelo(arquivo_filtrado:DataFrame, subsistema:str) -> str:
    """Gera o dado em um modelo padrão por subsistema.

    Args:
        arquivo_filtrado (DataFrame): dataframe com os dados previamente tratados.
        subsistema (str): [seco, n, ne, s e imperatriz] subsistema de interesse. 1 por vez.

    Returns:
        str: dado formatado
    """
    dicionario = {}
    
    # Tratativa de erro
    subsistemas_aceitos = ['seco', 'n', 'ne', 's', 'imperatriz']
    subsistema = subsistema if subsistema in subsistemas_aceitos else sys.exit("Subsistema inválido. Utilize ['seco', 'n', 'ne', 's', 'imperatriz']")
    
    # Pegando colunas necessários
    dados = arquivo_filtrado[['subsistema','data', 'demanda']]
    
    # filtrando subsistema
    teste = dados[dados['subsistema'] == subsistema]
    
    # seleção dos dados
    teste = teste[['demanda', 'data']]
    valores = teste.to_dict('records')
    
    # gerando formatação
    dicionario["subsistema"] = subsistema
    dicionario["data_partida"] = hoje.to_datetime_string()
    dicionario["valores"] = valores
    
    return dicionario

In [11]:
gera_modelo(arquivo_filtrado, 's')

{'subsistema': 's',
 'data_partida': '2021-10-03 00:00:00',
 'valores': [{'demanda': '10126', 'data': Timestamp('2021-10-02 00:00:00')},
  {'demanda': '9747', 'data': Timestamp('2021-10-02 00:30:00')},
  {'demanda': '9442', 'data': Timestamp('2021-10-02 01:00:00')},
  {'demanda': '9228', 'data': Timestamp('2021-10-02 01:30:00')},
  {'demanda': '9047', 'data': Timestamp('2021-10-02 02:00:00')},
  {'demanda': '8923', 'data': Timestamp('2021-10-02 02:30:00')},
  {'demanda': '8870', 'data': Timestamp('2021-10-02 03:00:00')},
  {'demanda': '8872', 'data': Timestamp('2021-10-02 03:30:00')},
  {'demanda': '8920', 'data': Timestamp('2021-10-02 04:00:00')},
  {'demanda': '8983', 'data': Timestamp('2021-10-02 04:30:00')},
  {'demanda': '9307', 'data': Timestamp('2021-10-02 05:00:00')},
  {'demanda': '9668', 'data': Timestamp('2021-10-02 05:30:00')},
  {'demanda': '10350', 'data': Timestamp('2021-10-02 06:00:00')},
  {'demanda': '10986', 'data': Timestamp('2021-10-02 06:30:00')},
  {'demanda': '1